In [8]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize
import random
import math

In [ ]:
#here we want to find the local maximum for a 2 variables function
#first we create a population of n individuals
#each individual is a x,y value randomly generated
#so the population is a n solutions for x and y
#our goal is to find an x,y solution that is pretty close to
#local maximum of the function which is a point p(x,y)

def generate_population(size,x_boundaries,y_boundaries):
    lower_x_boundry,upper_x_boundry=x_boundaries
    lower_y_boundry,upper_y_boundry=y_boundaries
    
    population=[]
    for i in range(size):
        individual={
            "x" : random.uniform(lower_x_boundry,upper_x_boundry),
            "y" : random.uniform(lower_y_boundry,upper_y_boundry)
        }
        population.append(individual)
    return population

#---------------------------------------------------------------------------------

def foo(x,y):
    return math.sin(math.sqrt(x**2+y**2))

def fitness_function(individual):
    x=individual['x']
    y=individual['y']
    return foo(x,y)

#---------------------------------------------------------------------------------

#this algorithme is one of many to the individuals to reproduce
#it takes as parameters the population sorted and the fitness sum of all individuals
#it returns the individual to reproduce

def choice_by_roulette(sorted_population, fitness_sum):
    offset = 0
    normalized_fitness_sum = fitness_sum

    lowest_fitness = fitness_function(sorted_population[0])
    if lowest_fitness < 0:
        offset = -lowest_fitness
        normalized_fitness_sum += offset * len(sorted_population)

    draw = random.uniform(0, 1)

    accumulated = 0
    for individual in sorted_population:
        fitness = fitness_function(individual) + offset
        probability = fitness / normalized_fitness_sum
        accumulated += probability

        if draw <= accumulated:
            return individual
        
#---------------------------------------------------------------------------------

#this apply the fitness function on each individual of the 
# population then sort the population algorithm sort the
#population according to the fitness values

def sort_population_by_fitness(population):
    return sorted(population, key=fitness_function)

#---------------------------------------------------------------------------------

#this function produces an individual child from two 
#individuals parent

def crossover(individual_a,individual_b):
    return {'x':(individual_a['x']+individual_b['x'])/2,
            'y':(individual_b['x']+individual_b['x'])/2}

#---------------------------------------------------------------------------------

#we mutate the new produced individual(kandiro lih tafra 
# bash ikon mbdel 3la parents) 

def mutate(individual):
    next_x=individual['x']+random.uniform(-0.05,0.05)
    next_y=individual['y']+random.uniform(-0.05,0.05)
    
    lower_boundary, upper_boundary=(-4,4)
    #guarentee we keep inside boundaries
    
    next_x=min(max(next_x, lower_boundary), upper_boundary)
    next_y=min(max(next_y, lower_boundary), upper_boundary)
    
    return {'x':next_x, 'y':next_y}

#---------------------------------------------------------------------------------

def make_next_generation(previous_population):
    next_generation=[]
    sorted_by_fitness_population=sort_population_by_fitness(previous_population)
    population_size=len(previous_population)
    fitness_sum=sum(fitness_function(individual) for individual in previous_population)
    
    for i in range(population_size):
        
        first_individual_parent=choice_by_roulette(sorted_by_fitness_population,fitness_sum)
        second_individual_parent=choice_by_roulette(sorted_by_fitness_population,fitness_sum)

        individual_child=crossover(first_individual_parent,second_individual_parent)
        #mutate the individual chils
        individual_child=mutate(individual_child)
        
        next_generation.append(individual_child)
        
    return next_generation

#---------------------------------------------------------------------------------

maximum_generations=1000
population=generate_population(10,(-4,4),(-4,4))

i=1
while True:
    print(f"🧬 GENERATION {i}")
    
    for individual in population:
        print(individual, fitness_function(individual))
    
    if i==maximum_generations:
        break
    
    i+=1
    
    
    population = make_next_generation(population)
    
best_individual = sort_population_by_fitness(population)[-1]
print("\n🔬 FINAL RESULT")
print(best_individual, fitness_function(best_individual))

In [ ]:
window=tk.Tk(className='Optimization avec GA')

window.geometry('530x300')

frame1=tk.Frame(window)
frame1.grid(row=0,column=0)

objectiveLabel=tk.Label(frame1,text='fonction objective : ')
objectiveLabel.grid(row=0,column=0,pady=10)

objectiveInput=tk.Entry(frame1)
objectiveInput.grid(row=0,column=1,columnspan=3,pady=10,ipadx=60)

contraintLabel=tk.Label(frame1,text='Contraint : ')
contraintLabel.grid(row=1,column=0,pady=10)

contrainInput1=tk.Entry(frame1)
contrainInput1.grid(row=1,column=1,pady=10)

contraintOp = tk.StringVar()
contraintOpSelect=ttk.Combobox(frame1,values=['=','>'],textvariable=contraintOp,width=5)
contraintOpSelect.grid(row=1,column=2,pady=10,padx=5)

contrainLabel2=tk.Label(frame1,text='0')
contrainLabel2.grid(row=1,column=3,pady=10,sticky='W')

XboundariesLbl=tk.Label(frame1,text='Les Bornes de x :')
XboundariesLbl.grid(row=2,column=0,pady=10)

XboundariesFrame=tk.Frame(frame1)

XupperBoundaryInpt=tk.Entry(XboundariesFrame,width=5)
XupperBoundaryInpt.grid(row=0,column=0,padx=5)

boundariesXlabel=tk.Label(XboundariesFrame,text='x')
boundariesXlabel.grid(row=0,column=1)

XlowerBoundaryInpt=tk.Entry(XboundariesFrame,width=5)
XlowerBoundaryInpt.grid(row=0,column=2,padx=5)

XboundariesFrame.grid(row=2,column=1,pady=10)

YboundariesLbl=tk.Label(frame1,text='Les Bornes de y :')
YboundariesLbl.grid(row=2,column=2,pady=10)

YboundariesFrame=tk.Frame(frame1)

YupperBoundaryInpt=tk.Entry(YboundariesFrame,width=5)
YupperBoundaryInpt.grid(row=0,column=0,padx=5)

YboundariesXlabel=tk.Label(YboundariesFrame,text='y')
YboundariesXlabel.grid(row=0,column=1)

YlowerBoundaryInpt=tk.Entry(YboundariesFrame,width=5)
YlowerBoundaryInpt.grid(row=0,column=2,padx=5)

YboundariesFrame.grid(row=2,column=3,pady=10)

popSizeLabel=tk.Label(frame1,text='taille du Population : ')
popSizeLabel.grid(row=3,column=0,pady=10)

popSize=tk.StringVar()
popSizeSelect=ttk.Combobox(frame1,values=[x for x in range(1,11)],textvariable=popSize,width=5)
popSizeSelect.grid(row=3,column=1,pady=10,padx=5)

IterNumLabel=tk.Label(frame1,text='Nombre D\'iterations : ')
IterNumLabel.grid(row=3,column=2,pady=10)

iterNum=tk.StringVar()
IterNumSelect=ttk.Combobox(frame1,values=[x for x in range(100,1001,100)],textvariable=iterNum,width=5)
IterNumSelect.grid(row=3,column=3,pady=10,padx=5)

CrossLabel=tk.Label(frame1,text='Probabilite du Crossver : ')
CrossLabel.grid(row=4,column=0,pady=10)

crossProb=tk.StringVar()
CrossSelect=ttk.Combobox(frame1,values=[x for x in range(1,10,1)],textvariable=crossProb,width=5)
CrossSelect.grid(row=4,column=1,pady=10,padx=5)

MutationLabel=tk.Label(frame1,text='Probabilite du Mutation : ')
MutationLabel.grid(row=4,column=2,pady=10)

mutationProb=tk.StringVar()
MutationSelect=ttk.Combobox(frame1,values=[float(f'0.{x}') for x in range(1,11)],textvariable=mutationProb,width=5)
MutationSelect.grid(row=4,column=3,pady=10,padx=5)

OpType=tk.IntVar()
MaxRadio=tk.Radiobutton(frame1,text='Maximisation',variable=OpType,value=1)
MaxRadio.grid(row=5,column=1)
MinRadio=tk.Radiobutton(frame1,text='Minimisation',variable=OpType,value=-1)
MinRadio.grid(row=5,column=2)
    

def plot(XBoundries,YBoundries,strObjective,Contraint,TypeContraint,OptimizationType):
    
    window.geometry('1060x650')
    txt="🧬 GENERATION 1\n\
{'x': 3.922262400256203, 'y': -3.1466348170967535} -0.9504626839618694    \n\
{'x': -0.4693503981479008, 'y': 1.449548155390156} 0.9988883588424676     \n\
{'x': 2.814552500012077, 'y': -0.7564318111520256} 0.22521521174276216    \n\
{'x': -2.2467555039073748, 'y': -0.6700508850371678} 0.7152979517218132   \n\
{'x': -2.8240677002765624, 'y': -0.8754803484714104} 0.18388268620452528  \n\
{'x': -1.149768781399791, 'y': 2.492229303023711} 0.3865885070250822      \n\
{'x': -2.310157332562028, 'y': -1.0578233154862344} 0.5652717576905796    \n\
{'x': -0.5974591387242363, 'y': -0.9723291148563344} 0.909141964224674    \n\
{'x': 3.4138411233481687, 'y': -3.661031958609379} -0.9572793046385382    \n\
{'x': -2.0918296294248346, 'y': 3.30809468175464} -0.69784735428813       \n\
🧬 GENERATION 2\n\
{'x': 3.922262400256203, 'y': -3.1466348170967535} -0.9504626839618694\n\
{'x': -0.4693503981479008, 'y': 1.449548155390156} 0.9988883588424676\n\
{'x': 2.814552500012077, 'y': -0.7564318111520256} 0.22521521174276216\n\
{'x': -2.2467555039073748, 'y': -0.6700508850371678} 0.7152979517218132\n\
{'x': -2.8240677002765624, 'y': -0.8754803484714104} 0.18388268620452528\n\
{'x': -1.149768781399791, 'y': 2.492229303023711} 0.3865885070250822\n\
{'x': -2.310157332562028, 'y': -1.0578233154862344} 0.5652717576905796\n\
{'x': -0.5974591387242363, 'y': -0.9723291148563344} 0.9091419642246741\n\
{'x': 3.4138411233481687, 'y': -3.661031958609379} -0.9572793046385382\n\
{'x': -2.0918296294248346, 'y': 3.30809468175464} -0.69784735428813\n\
"

    frame3.grid(row=0,column=1)
    traitmentLabel=tk.Label(frame3,text=txt,bg='white')
    traitmentLabel.grid(row=0,column=0,ipadx=40,padx=10,pady=10)
    
    newCon=Contraint
    if newCon.find('exp')==-1 :
        newCon=newCon.replace('x','x[0]')
    else:
        for i in range(len(newCon)):
            if newCon[i]=='x':
                if newCon[i-1]!='e':
                    newCon=list(newCon)
                    newCon[i]='x[0]'
                    newCon=''.join(newCon)
    newCon=newCon.replace('y','x[1]')
    
    newObj=strObjective
    if strObjective.find('exp')==-1 :
        newObj=newObj.replace('x','x[0]')
    else:
        for i in range(len(newObj)):
            if newObj[i]=='x':
                if newObj[i-1]!='e':
                    newObj=list(newObj)
                    newObj[i]='x[0]'
                    newObj=''.join(newObj)
    newObj=newObj.replace('y','x[1]')
    
    newObj=f'({OptimizationType})*({newObj})'
    
    def optType(OptimizationType):
        if OptimizationType==1:
            return 'Max'
        return 'Min'
    
    def objective(x,newObj):
        return eval(newObj)
    
    def constraint(x):
        return eval(newCon)
    
    x=[0,0]
    cons=[{'type':TypeContraint,'fun':constraint}]
    if XBoundries==False:
        sol=minimize(objective,
                 x,
                 args=(newObj),
                 method='SLSQP',
                 constraints=cons
                )
    else:
        sol=minimize(objective,
                     x,
                     args=(newObj),
                     method='SLSQP',
                     bounds=[(XBoundries[0],XBoundries[1]),(YBoundries[0],YBoundries[1])],
                     constraints=cons
                    )
    if sol.message=='Iteration limit reached':
        newObj='(-1)*('+newObj+')'
        sol=minimize(objective,
                     x,
                     args=(newObj),
                     method='SLSQP',
                     constraints=cons
                    )
        sol.jac=sol.jac*(-1)
        
    print('objectice : '+strObjective+' new Objective : '+f'{newObj}')
    print('contraint :'+Contraint+'new Contraint : '+newCon)
    print('typecon   : '+TypeContraint)
    print("optype : "+str(OptimizationType))
    print(sol)
    global resultLabel
    global counter
    resultLabel=tk.Label(frame2,text=f'{optType(OptimizationType)}(x,y)=({sol.jac[0]} , {sol.jac[1]})')
    resultLabel.grid(row=0,column=0)
    counter=1
    
    if XBoundries==False:
        XBoundries=np.array([-4,4])
        YBoundries=np.array([-4,4])
    
    x = np.linspace(XBoundries[0],XBoundries[1],30)
    y = np.linspace(YBoundries[0],YBoundries[1],30)

    x, y = np.meshgrid(x, y)
    Z = eval(strObjective)
    
    fig = Figure(figsize = (4, 3),dpi = 100)
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(x, y, Z, rstride=1, cstride=1, cmap=cm.jet, linewidth=0)
    fig.colorbar(surf)
    title = ax.set_title("Title")
    title.set_y(1.01)

    ax.xaxis.set_major_locator(MaxNLocator(5))
    ax.yaxis.set_major_locator(MaxNLocator(6))
    ax.zaxis.set_major_locator(MaxNLocator(5))

    fig.tight_layout()
    
    #add figure to frame
    canvas = FigureCanvasTkAgg(fig,master = frame2)  
    canvas.draw()
    canvas.get_tk_widget().grid(row=1,column=0)

def DetConType(constStr):
    if constStr=='=':
        return 'eq'
    else:
        return 'ineq'

counter=0 

def clearLabel():
    if counter != 0:
        resultLabel.destroy()
        
def retrieve():
    strObjective=objectiveInput.get()
    Contraint=contrainInput1.get()
    TypeContraint=DetConType(contraintOp.get())
    try:
        XBoundries=np.array([float(XupperBoundaryInpt.get()),float(XlowerBoundaryInpt.get())])
        YBoundries=np.array([float(YupperBoundaryInpt.get()),float(YlowerBoundaryInpt.get())])
    except:
        XBoundries=False
        YBoundries=False
    
#     PopulationSize=float(popSize.get())
#     Iterations=int(iterNum.get())
#     crossoverProb=float(crossProb.get())
#     mutationProba=float(mutationProb.get())
    OptimizationType=OpType.get()
    plot(XBoundries,YBoundries,strObjective,Contraint,TypeContraint,OptimizationType)


solveBtn=tk.Button(frame1,text='Solve',activebackground='#119911',command=lambda:[clearLabel(),retrieve()])
solveBtn.grid(row=6,columnspan=4,pady=20,ipadx=20)

frame2=tk.Frame(window)
frame2.grid(row=1,column=0)

frame3=tk.Frame(window)

window.mainloop()

objectice : x+y new Objective : (1)*(x[0]+x[1])
contraint :x+y-2new Contraint : x[0]+x[1]-2
typecon   : eq
optype : 1
     fun: 1.9999999999999996
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([1., 1.])
